In [4]:
import numpy as np
from sklearn.cluster import SpectralClustering
import skfuzzy as fuzz

# Set random seed for reproducibility
np.random.seed(42)

# Simulation parameters
num_providers = 3
num_regions = 3
min_dcs_per_provider = 2
max_dcs_per_provider = 5
num_vms_per_dc = 8
task_counts = [1000, 2000, 3000, 5000, 7000, 10000]
task_size_range = (200, 1000)
data_size_range = (300, 1000)
inter_region_bw = 500  # in Mb/s
inter_region_delay = 150  # in ms
intra_region_bw = 1000  # in Mb/s
intra_region_delay = 50  # in ms
intra_dc_bw = 8000  # in Mb/s
intra_dc_delay = 10  # in ms
vm_processing_capability = 1500  # in MIPS
vm_num_cpus = 2
vm_ram = 4  # in Gb
vm_storage_capacity = 8  # in Gb
revenue_per_task_execution = 0.7  # in $
penalty_per_violation = 0.0025  # in $
response_time_slo = 180  # in seconds
availability_slo = 0.95

# Initialize empty lists to store simulated data
provider_data = []

# Set random seed for reproducibility
np.random.seed(42)

# CPU Price ($/107MI), Storage Price ($/Gb), BW Price ($/Gb)
provider_pricing = {
    'Provider 1': {
        'US': [0.020, 0.006, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.025, 0.006, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.027, 0.0066, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
    'Provider 2': {
        'US': [0.020, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.018, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.020, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
    'Provider 3': {
        'US': [0.0095, 0.00120, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.0090, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.0080, 0.0090, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
}

# Simulate data for each provider and region
provider_data = []
for provider_id in range(1, num_providers + 1):
    num_dcs = np.random.randint(min_dcs_per_provider, max_dcs_per_provider + 1)
    for region in ['US', 'EU', 'AS']:
        for dc_id in range(1, num_dcs + 1):
            vm_data = {
                'provider_id': provider_id,
                'region': region,
                'dc_id': dc_id,
                'num_vms': num_vms_per_dc,
                'vm_processing_capability': vm_processing_capability,
                'vm_num_cpus': vm_num_cpus,
                'vm_ram': vm_ram,
                'vm_storage_capacity': vm_storage_capacity,
                'cpu_price': provider_pricing[f'Provider {provider_id}'][region][0],
                'storage_price': provider_pricing[f'Provider {provider_id}'][region][1],
                'intra_dc_bw_price': provider_pricing[f'Provider {provider_id}'][region][2],
                'intra_dc_bw_price': provider_pricing[f'Provider {provider_id}'][region][3],
                'intra_dc_bw_price': provider_pricing[f'Provider {provider_id}'][region][4],
                'inter_region_bw_price': provider_pricing[f'Provider {provider_id}'][region][5],
            }
            provider_data.append(vm_data)

# Assuming 'provider_data' includes task-related features
features = [
    'cpu_price', 'storage_price', 'intra_dc_bw_price', 'inter_region_bw_price',
    'total_cost', 'task_count', 'task_size', 'response_time_slo', 'availability_slo'
]
data = np.array([[vm.get(feature, 0) for feature in features] for vm in provider_data])

# Data Identification Phase: Spectral Clustering
n_clusters = 3  # Adjust based on your data
spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', n_neighbors=5)
cluster_labels = spectral.fit_predict(data)

# Fuzzy Inference System
# Define fuzzy input variables
data_transfer_time_ratio = np.arange(0, 1.1, 0.1)
virtual_machine_load = np.arange(0, 1.1, 0.1)
data_availability = np.arange(0, 1.1, 0.1)

# Output variables
provider_profit = np.arange(0, 1.1, 0.1)
placement_potential = np.arange(0, 1.1, 0.1)

# Membership functions
high_mf = fuzz.trimf(np.arange(0, 1.1, 0.1), [0.7, 1, 1])
medium_mf = fuzz.trimf(np.arange(0, 1.1, 0.1), [0.3, 0.5, 0.7])
low_mf = fuzz.trimf(np.arange(0, 1.1, 0.1), [0, 0, 0.3])
nr_mf = fuzz.trimf(np.arange(0, 1.1, 0.1), [0, 0, 0.5])
r_mf = fuzz.trimf(np.arange(0, 1.1, 0.1), [0.5, 1, 1])
very_low_mf = fuzz.trimf(np.arange(0, 1.1, 0.1), [0, 0, 0.2])
low_mf = fuzz.trimf(np.arange(0, 1.1, 0.1), [0.2, 0.4, 0.6])
medium_mf = fuzz.trimf(np.arange(0, 1.1, 0.1), [0.4, 0.6, 0.8])
high_mf = fuzz.trimf(np.arange(0, 1.1, 0.1), [0.6, 0.8, 1])
very_high_mf = fuzz.trimf(np.arange(0, 1.1, 0.1), [0.8, 1, 1])

# Apply the membership functions to input and output variables
data_transfer_time_ratio_high = high_mf
data_transfer_time_ratio_medium = medium_mf
data_transfer_time_ratio_low = low_mf

virtual_machine_load_high = high_mf
virtual_machine_load_medium = medium_mf
virtual_machine_load_low = low_mf

data_availability_nr = nr_mf
data_availability_r = r_mf

provider_profit_very_low = very_low_mf
provider_profit_low = low_mf
provider_profit_medium = medium_mf
provider_profit_high = high_mf
provider_profit_very_high = very_high_mf

placement_potential_very_low = very_low_mf
placement_potential_low = low_mf
placement_potential_medium = medium_mf
placement_potential_high = high_mf
placement_potential_very_high = very_high_mf

# Fuzzy inference rules
rule1 = np.fmin(np.fmin(data_transfer_time_ratio_high, virtual_machine_load_high), data_availability_nr)
rule2 = np.fmin(np.fmin(data_transfer_time_ratio_high, virtual_machine_load_medium), data_availability_nr)
rule3 = np.fmin(np.fmin(data_transfer_time_ratio_medium, virtual_machine_load_medium), data_availability_nr)
rule4 = np.fmin(np.fmin(data_transfer_time_ratio_medium, virtual_machine_load_medium), data_availability_r)
rule5 = np.fmin(np.fmin(data_transfer_time_ratio_low, virtual_machine_load_low), data_availability_r)

# Aggregate the rules
aggregated = np.fmax(rule1, np.fmax(rule2, np.fmax(rule3, np.fmax(rule4, rule5))))

# Defuzzify the aggregated result
placement_potential_crisp = fuzz.defuzz(placement_potential, aggregated, 'centroid')

print("Cluster Labels:", cluster_labels)
print("Placement Potential:", placement_potential_crisp)



Cluster Labels: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 2 2 2 2 2 2]
Placement Potential: 0.6666666666666666


c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
